In [ ]:
import pandas as pd
df = pd.read_csv("/content/engineered_df.csv")

In [ ]:
df.isnull().sum()

Unnamed: 0              0
id                      0
qid1                    0
qid2                    0
question1               1
question2               0
is_duplicate            0
q1_len                  0
q2_len                  0
q1_num_words            0
q2_num_words            0
word_common             0
total_words             0
word_share              0
cwc_min                 0
cwc_max                 0
csc_min                 0
csc_max                 0
ctc_min                 0
ctc_max                 0
last_word_eq            0
first_word_eq           0
abs_len_diff            0
mean_len                0
longest_substr_ratio    0
fuzz_ratio              0
fuzz_partial_ratio      0
token_sort_ratio        0
token_set_ratio         0
dtype: int64

In [ ]:
df.dropna(subset=['question1'],inplace=True)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
# merge texts
questions = list(df['question1']) + list(df['question2'])

tfidf = TfidfVectorizer(max_features=4000)
q1_vec, q2_vec = np.vsplit(tfidf.fit_transform(questions).toarray(),2)

In [ ]:
temp_df1 = pd.DataFrame(q1_vec, index= df.index)
temp_df2 = pd.DataFrame(q2_vec, index= df.index)
temp_df = pd.concat([temp_df1, temp_df2], axis=1)
temp_df.shape

(29999, 8000)

In [ ]:
final_df = pd.concat([temp_df, df.drop(columns = ["id","qid1","qid2","question1","question2"])], axis=1)
print(final_df.shape)
final_df.head()

(29999, 8024)


,0,1,2,3,4,5,6,7,8,9,...,ctc_max,last_word_eq,first_word_eq,abs_len_diff,mean_len,longest_substr_ratio,fuzz_ratio,fuzz_partial_ratio,token_sort_ratio,token_set_ratio
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.499996,0.0,1.0,4.0,12.0,0.340426,67,76,65,82
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.272725,0.0,0.0,3.0,9.5,0.261905,65,72,54,61
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.285713,1.0,1.0,11.0,15.5,0.230769,51,49,50,71
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.846147,0.0,1.0,1.0,12.5,0.576923,85,84,83,95
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.714276,1.0,1.0,1.0,6.5,0.321429,68,63,88,96


In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(final_df.drop("is_duplicate",axis=1).values,final_df["is_duplicate"].values,test_size=0.2,random_state=1)


In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [ ]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
xgb = XGBClassifier()
xgb.fit(X_train,y_train)
y_pred1 = xgb.predict(X_test)
accuracy_score(y_test,y_pred1)

0.7845

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,y_pred1)

array([[3202,  565],
       [ 728, 1505]])

In [ ]:
import pickle

pickle.dump(xgb,open('model.pkl','wb'))
pickle.dump(tfidf,open('tfidf.pkl','wb'))